# RT-DETR Pretraining with SHIFT-Discrete Dataset

In [1]:
import os
os.environ["WANDB_DISABLED"] = "true"

## Check GPU Availability

In [2]:
!nvidia-smi

Sun Aug 24 22:04:09 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.183.06             Driver Version: 535.183.06   CUDA Version: 12.4     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100 80GB PCIe          On  | 00000000:17:00.0 Off |                    0 |
| N/A   25C    P0              43W / 300W |      0MiB / 81920MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [3]:
# Set CUDA Device Number
DEVICE_NUM = 0
ADDITIONAL_GPU = 0

from os import environ
environ["CUDA_VISIBLE_DEVICES"] = ",".join([f"{i+DEVICE_NUM}" for i in range(0, ADDITIONAL_GPU+1)])
environ["CUDA_VISIBLE_DEVICES"]

'0'

## Imports

In [4]:
from os import path

import torch
from torch.utils.data import DataLoader

from ttadapters.datasets import BaseDataset, DatasetHolder, DataLoaderHolder
from ttadapters.datasets import SHIFTClearDatasetForObjectDetection, SHIFTCorruptedDatasetForObjectDetection
from transformers import Trainer, TrainingArguments, EarlyStoppingCallback
from accelerate import Accelerator, notebook_launcher

from supervision.metrics.mean_average_precision import MeanAveragePrecision
from supervision.detection.core import Detections

#import wandb
from tqdm.auto import tqdm
import matplotlib.pyplot as plt

In [5]:
if torch.cuda.is_available():
    if ADDITIONAL_GPU:
        device = torch.device("cuda")
    else:
        device = torch.device(f"cuda")  # torch.device(f"cuda:{DEVICE_NUM}")
else:
    device = torch.device("cpu")
    DEVICE_NUM = -1

print(f"INFO: Using device - {device}" + (f":{DEVICE_NUM}" if ADDITIONAL_GPU else ""))

INFO: Using device - cuda


In [6]:
# Tqdm Test
for _ in tqdm(range(100)):
    pass

  0%|          | 0/100 [00:00<?, ?it/s]

In [7]:
PROJECT_NAME = "APT_SHIFT_Pretraining"
RUN_NAME = "RT-DETR_R50"

# WandB Initialization
#wandb.init(project=PROJECT_NAME, name=RUN_NAME)

## Define Dataset

In [8]:
DATA_ROOT = path.join(".", "data")

dataset = DatasetHolder(
    train=SHIFTClearDatasetForObjectDetection(root=DATA_ROOT, train=True),
    valid=SHIFTClearDatasetForObjectDetection(root=DATA_ROOT, valid=True),
    test=SHIFTCorruptedDatasetForObjectDetection(root=DATA_ROOT, valid=True)
)

[08/24/2025 22:04:18] SHIFT DevKit - INFO - Base: ./data/SHIFT/discrete/images/train. Backend: <shift_dev.utils.backend.ZipBackend object at 0x7fe5c4f18440>
[08/24/2025 22:04:18] SHIFT DevKit - INFO - Loading annotation from './data/SHIFT_SUBSET/normal/discrete/images/train/front/det_2d.json' ...


INFO: Downloading 'SHIFT_SUBSET' from file server to ./data/SHIFT/discrete...
INFO: Dataset archive found in the root directory. Skipping download.
INFO: Subset split for 'SHIFT_SUBSET' dataset is already done. Skipping...
INFO: Downloading 'SHIFT_SUBSET' from file server to ./data/SHIFT/discrete...
INFO: Dataset archive found in the root directory. Skipping download.


[08/24/2025 22:04:19] SHIFT DevKit - INFO - Loading annotation from './data/SHIFT_SUBSET/normal/discrete/images/train/front/det_2d.json' Done.
[08/24/2025 22:04:24] SHIFT DevKit - INFO - Loading annotation takes 5.70 seconds.


Batch 0:

Item                 Shape                               Min        Max       
--------------------------------------------------------------------------------
original_hw          [tensor([800]), tensor([1280])]
input_hw             [tensor([800]), tensor([1280])]
frame_ids            torch.Size([1])                           0.00       0.00
name                 ['00000000_img_front.jpg']
videoName            ['0016-1b62']
intrinsics           torch.Size([1, 3, 3])                     0.00     640.00
extrinsics           torch.Size([1, 4, 4])                    -7.53     219.91
boxes2d              torch.Size([1, 26, 4])                    5.00     974.00
boxes2d_classes      torch.Size([1, 26])                       0.00       3.00
boxes2d_track_ids    torch.Size([1, 26])                       0.00      25.00
images               torch.Size([1, 1, 3, 800, 1280])          0.00     255.00



[08/24/2025 22:04:26] SHIFT DevKit - INFO - Base: ./data/SHIFT/discrete/images/val. Backend: <shift_dev.utils.backend.ZipBackend object at 0x7fe5c4f18440>
[08/24/2025 22:04:26] SHIFT DevKit - INFO - Loading annotation from './data/SHIFT_SUBSET/normal/discrete/images/val/front/det_2d.json' ...
[08/24/2025 22:04:26] SHIFT DevKit - INFO - Loading annotation from './data/SHIFT_SUBSET/normal/discrete/images/val/front/det_2d.json' Done.


Video name: 0016-1b62
Sample indices within a video: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]
INFO: Downloading 'SHIFT_SUBSET' from file server to ./data/SHIFT/discrete...
INFO: Dataset archive found in the root directory. Skipping download.
INFO: Subset split for 'SHIFT_SUBSET' dataset is already done. Skipping...
INFO: Downloading 'SHIFT_SUBSET' from file server to ./data/SHIFT/discrete...
INFO: Dataset archive found in the root directory. Skipping download.


[08/24/2025 22:04:26] SHIFT DevKit - INFO - Loading annotation takes 0.61 seconds.
[08/24/2025 22:04:27] SHIFT DevKit - INFO - Base: ./data/SHIFT/discrete/images/val. Backend: <shift_dev.utils.backend.ZipBackend object at 0x7fe5c4f18440>
[08/24/2025 22:04:27] SHIFT DevKit - INFO - Loading annotation from './data/SHIFT_SUBSET/corrupted/discrete/images/val/front/det_2d.json' ...


Batch 0:

Item                 Shape                               Min        Max       
--------------------------------------------------------------------------------
original_hw          [tensor([800]), tensor([1280])]
input_hw             [tensor([800]), tensor([1280])]
frame_ids            torch.Size([1])                           0.00       0.00
name                 ['00000000_img_front.jpg']
videoName            ['0116-4859']
intrinsics           torch.Size([1, 3, 3])                     0.00     640.00
extrinsics           torch.Size([1, 4, 4])                    -0.90     138.34
boxes2d              torch.Size([1, 6, 4])                   246.00     859.00
boxes2d_classes      torch.Size([1, 6])                        1.00       5.00
boxes2d_track_ids    torch.Size([1, 6])                        0.00       5.00
images               torch.Size([1, 1, 3, 800, 1280])          0.00     255.00

Video name: 0116-4859
Sample indices within a video: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10,

[08/24/2025 22:04:28] SHIFT DevKit - INFO - Loading annotation from './data/SHIFT_SUBSET/corrupted/discrete/images/val/front/det_2d.json' Done.
[08/24/2025 22:04:33] SHIFT DevKit - INFO - Loading annotation takes 6.36 seconds.


Batch 0:

Item                 Shape                               Min        Max       
--------------------------------------------------------------------------------
original_hw          [tensor([800]), tensor([1280])]
input_hw             [tensor([800]), tensor([1280])]
frame_ids            torch.Size([1])                           0.00       0.00
name                 ['00000000_img_front.jpg']
videoName            ['007b-4e72']
intrinsics           torch.Size([1, 3, 3])                     0.00     640.00
extrinsics           torch.Size([1, 4, 4])                  -311.22     226.46
boxes2d              torch.Size([1, 3, 4])                   233.00     802.00
boxes2d_classes      torch.Size([1, 3])                        0.00       1.00
boxes2d_track_ids    torch.Size([1, 3])                        0.00       2.00
images               torch.Size([1, 1, 3, 800, 1280])          0.00     255.00

Video name: 007b-4e72
Sample indices within a video: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10,

In [9]:
dataset.train[1]['front'].keys()

dict_keys(['original_hw', 'input_hw', 'frame_ids', 'name', 'videoName', 'intrinsics', 'extrinsics', 'boxes2d', 'boxes2d_classes', 'boxes2d_track_ids', 'images'])

In [10]:
dataset.train[999]

{'front': {'original_hw': (800, 1280),
  'input_hw': (800, 1280),
  'frame_ids': 490,
  'name': '00000490_img_front.jpg',
  'videoName': '0c9d-eefc',
  'intrinsics': tensor([[640.,   0., 640.],
          [  0., 640., 400.],
          [  0.,   0.,   1.]]),
  'extrinsics': tensor([[-5.7429e-01,  7.7804e-01, -2.5465e-01,  1.6100e+02],
          [-7.0979e-01, -6.2821e-01, -3.1867e-01, -2.0023e+01],
          [-4.0791e-01, -2.2626e-03,  9.1302e-01,  1.5929e+00],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  1.0000e+00]]),
  'boxes2d': tensor([[ 457.,  405.,  525.,  467.],
          [ 599.,  391.,  612.,  403.],
          [ 599.,  398.,  677.,  459.],
          [ 835.,  391., 1280.,  605.],
          [ 655.,  396.,  668.,  402.],
          [ 392.,  394.,  404.,  401.],
          [ 665.,  396.,  676.,  402.],
          [ 842.,  390.,  848.,  397.],
          [1207.,  380., 1217.,  399.]]),
  'boxes2d_classes': tensor([1, 2, 1, 1, 1, 2, 1, 0, 0]),
  'boxes2d_track_ids': tensor([ 4,  1,  0

In [11]:
dataset.train[1000]['front']['images'].shape  # should be (batch_size, num_channels, height, width)

torch.Size([1, 3, 800, 1280])

## DataLoader

In [12]:
# Set Batch Size
BATCH_SIZE = 2, 8, 8, 8  # 4070 Ti
BATCH_SIZE = 32, 64, 64, 32  # A6000
BATCH_SIZE = 32, 32, 32, 32  # A6000

# Dataset Configs
CLASSES = dataset.train.classes
NUM_CLASSES = len(CLASSES)

print(f"INFO: Set batch size - Train: {BATCH_SIZE[0]}, Valid: {BATCH_SIZE[1]}, Test: {BATCH_SIZE[2]}")
print(f"INFO: Number of classes - {NUM_CLASSES} {CLASSES}")

INFO: Set batch size - Train: 32, Valid: 32, Test: 32
INFO: Number of classes - 6 ['pedestrian', 'car', 'truck', 'bus', 'motorcycle', 'bicycle']


In [13]:
class DatasetAdapterForTransformers(BaseDataset):
    def __init__(self, original_dataset, camera='front'):
        self.dataset = original_dataset
        self.camera = camera

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        item = self.dataset[idx][self.camera]
        image = item['images'].squeeze(0)

        # Convert to COCO_Detection Format
        annotations = []
        target = dict(image_id=idx, annotations=annotations)
        for box, cls in zip(item['boxes2d'], item['boxes2d_classes']):
            x1, y1, x2, y2 = box.tolist()  # from Pascal VOC format (x1, y1, x2, y2)
            width, height = x2 - x1, y2 - y1
            annotations.append(dict(
                bbox=[x1, y1, width, height],  # to COCO format: [x, y, width, height]
                category_id=cls.item(),
                area=width * height,
                iscrowd=0
            ))

        # Following prepare_coco_detection_annotation's expected format
        # RT-DETR ImageProcessor converts the COCO bbox to center format (cx, cy, w, h) during preprocessing
        # But, eventually re-converts the bbox to Pascal VOC (x1, y1, x2, y2) format after post-processing
        return dict(image=image, target=target)

In [14]:
def collate_fn(batch, preprocessor=None):
    images = [item['image'] for item in batch]
    if preprocessor is not None:
        target = [item['target'] for item in batch]
        return preprocessor(images=images, annotations=target, return_tensors="pt")
    else:
        # If no preprocessor is provided, just assume images are already in tensor format
        return dict(
            pixel_values=dict(pixel_values=torch.stack(images)),
            labels=[dict(
                class_labels=item['boxes2d_classes'].long(),
                boxes=item["boxes2d"].float()
            ) for item in batch]
        )

## Load Model

In [15]:
from transformers import AutoBackbone, RTDetrForObjectDetection, RTDetrImageProcessorFast, RTDetrConfig, SwinConfig, ResNetConfig
from transformers.image_utils import AnnotationFormat

In [16]:
USE_PRETRAINED_MODEL = True
LOAD_ONLY_COCO_BACKBONE = False
USE_SWIN_T_BACKBONE = False
USE_SHIFT_BACKBONE = False

In [17]:
reference_model_id = "PekingU/rtdetr_r50vd"

# Load the reference model configuration
reference_config = RTDetrConfig.from_pretrained(reference_model_id, torch_dtype=torch.float32, return_dict=True)
reference_config.num_labels = NUM_CLASSES

# Set the image size and preprocessor size
reference_config.image_size = 800

# Load the reference model image processor
reference_preprocessor = RTDetrImageProcessorFast.from_pretrained(reference_model_id)
reference_preprocessor.format = AnnotationFormat.COCO_DETECTION  # COCO Format / Detection BBOX Format
reference_preprocessor.size = {"height": 800, "width": 800}
reference_preprocessor.do_resize = False

In [18]:
if USE_PRETRAINED_MODEL:
    # Load the pre-trained model
    model = RTDetrForObjectDetection.from_pretrained(reference_model_id, config=reference_config, torch_dtype=torch.float32, ignore_mismatched_sizes=True)
    if LOAD_ONLY_COCO_BACKBONE:
        detector_state = RTDetrForObjectDetection(config=reference_config).state_dict()
        detector_state = {k: v for k, v in model.state_dict().items() if 'backbone' not in k}
        model.load_state_dict(detector_state, strict=False)
else:
    # Set the backbone configuration
    if USE_SHIFT_BACKBONE:
        if USE_SWIN_T_BACKBONE:
            backbone_url = "https://github.com/robustaim/ContinualTTA_ObjectDetection/releases/download/backbone_converted/swin_tiny_patch4_window7_shift_from_detectron2.pth"
            reference_config.backbone_config = SwinConfig(
                embed_dim=96, depths=[2, 2, 6, 2], num_heads=[3, 6, 12, 24], output_hidden_states=True,
                window_size=7, out_features=["stage1", "stage2", "stage3"]  # stride 8·16·32
            )
        else:
            backbone_url = "https://github.com/robustaim/ContinualTTA_ObjectDetection/releases/download/backbone_converted/resnet50_shift_from_detectron2.pth"
            reference_config.backbone_config = ResNetConfig(depths=[3,4,6,3], out_indices=(2, 3, 4))
    else:
        if USE_SWIN_T_BACKBONE:
            backbone_id = "microsoft/swin-tiny-patch4-window7-224"
            reference_config.backbone_config = SwinConfig.from_pretrained(
                backbone_id, output_hidden_states=True, out_features=["stage1", "stage2", "stage3"]
            )
        else:
            backbone_id = "microsoft/resnet-50"
            reference_config.backbone_config = ResNetConfig.from_pretrained(backbone_id, out_indices=(2, 3, 4))

    # Initialize a new model with the reference configuration
    model = RTDetrForObjectDetection(config=reference_config)
    if USE_SHIFT_BACKBONE:
        backbone_state = torch.hub.load_state_dict_from_url(backbone_url, map_location="cpu")
        model.model.backbone.model.load_state_dict(backbone_state, strict=False)
    else:
        backbone_state = AutoBackbone.from_pretrained(backbone_id, config=reference_config.backbone_config).state_dict()
        model.model.backbone.model.load_state_dict(backbone_state, strict=False)
        if USE_SWIN_T_BACKBONE:
            model.model.backbone.model.forward.__kwdefaults__['interpolate_pos_encoding'] = True
    del backbone_state

model.to(device)

Some weights of RTDetrForObjectDetection were not initialized from the model checkpoint at PekingU/rtdetr_r50vd and are newly initialized because the shapes did not match:
- model.decoder.class_embed.0.bias: found shape torch.Size([80]) in the checkpoint and torch.Size([6]) in the model instantiated
- model.decoder.class_embed.0.weight: found shape torch.Size([80, 256]) in the checkpoint and torch.Size([6, 256]) in the model instantiated
- model.decoder.class_embed.1.bias: found shape torch.Size([80]) in the checkpoint and torch.Size([6]) in the model instantiated
- model.decoder.class_embed.1.weight: found shape torch.Size([80, 256]) in the checkpoint and torch.Size([6, 256]) in the model instantiated
- model.decoder.class_embed.2.bias: found shape torch.Size([80]) in the checkpoint and torch.Size([6]) in the model instantiated
- model.decoder.class_embed.2.weight: found shape torch.Size([80, 256]) in the checkpoint and torch.Size([6, 256]) in the model instantiated
- model.decoder.cl

RTDetrForObjectDetection(
  (model): RTDetrModel(
    (backbone): RTDetrConvEncoder(
      (model): RTDetrResNetBackbone(
        (embedder): RTDetrResNetEmbeddings(
          (embedder): Sequential(
            (0): RTDetrResNetConvLayer(
              (convolution): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
              (normalization): RTDetrFrozenBatchNorm2d()
              (activation): ReLU()
            )
            (1): RTDetrResNetConvLayer(
              (convolution): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
              (normalization): RTDetrFrozenBatchNorm2d()
              (activation): ReLU()
            )
            (2): RTDetrResNetConvLayer(
              (convolution): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
              (normalization): RTDetrFrozenBatchNorm2d()
              (activation): ReLU()
            )
          )
          (pooler): MaxPool2d(

In [19]:
test_d = DatasetAdapterForTransformers(dataset.train)[5]
test_d

{'image': tensor([[[ 64.,  65.,  67.,  ..., 121., 121., 121.],
          [ 62.,  61.,  64.,  ..., 121., 121., 121.],
          [ 58.,  60.,  61.,  ..., 121., 121., 121.],
          ...,
          [105., 105., 105.,  ...,  60.,  64.,  72.],
          [103., 103., 103.,  ...,  67.,  67.,  67.],
          [100., 100., 100.,  ...,  76.,  72.,  66.]],
 
         [[ 27.,  26.,  26.,  ..., 139., 139., 139.],
          [ 25.,  24.,  22.,  ..., 139., 139., 139.],
          [ 22.,  23.,  22.,  ..., 139., 139., 139.],
          ...,
          [106., 106., 106.,  ...,  59.,  63.,  71.],
          [104., 104., 104.,  ...,  66.,  66.,  66.],
          [101., 101., 101.,  ...,  75.,  71.,  65.]],
 
         [[  9.,   9.,   8.,  ..., 153., 153., 153.],
          [  7.,   6.,   6.,  ..., 153., 153., 153.],
          [  6.,   7.,   7.,  ..., 153., 153., 153.],
          ...,
          [111., 111., 111.,  ...,  65.,  69.,  77.],
          [109., 109., 109.,  ...,  72.,  72.,  72.],
          [106., 106.,

In [20]:
reference_preprocessor(images=test_d['image'], annotations=test_d['target'])

{'pixel_values': tensor([[[[0.2510, 0.2549, 0.2627,  ..., 0.4745, 0.4745, 0.4745],
          [0.2431, 0.2392, 0.2510,  ..., 0.4745, 0.4745, 0.4745],
          [0.2275, 0.2353, 0.2392,  ..., 0.4745, 0.4745, 0.4745],
          ...,
          [0.4118, 0.4118, 0.4118,  ..., 0.2353, 0.2510, 0.2824],
          [0.4039, 0.4039, 0.4039,  ..., 0.2627, 0.2627, 0.2627],
          [0.3922, 0.3922, 0.3922,  ..., 0.2980, 0.2824, 0.2588]],

         [[0.1059, 0.1020, 0.1020,  ..., 0.5451, 0.5451, 0.5451],
          [0.0980, 0.0941, 0.0863,  ..., 0.5451, 0.5451, 0.5451],
          [0.0863, 0.0902, 0.0863,  ..., 0.5451, 0.5451, 0.5451],
          ...,
          [0.4157, 0.4157, 0.4157,  ..., 0.2314, 0.2471, 0.2784],
          [0.4078, 0.4078, 0.4078,  ..., 0.2588, 0.2588, 0.2588],
          [0.3961, 0.3961, 0.3961,  ..., 0.2941, 0.2784, 0.2549]],

         [[0.0353, 0.0353, 0.0314,  ..., 0.6000, 0.6000, 0.6000],
          [0.0275, 0.0235, 0.0235,  ..., 0.6000, 0.6000, 0.6000],
          [0.0235, 0.0275

In [21]:
from transformers.trainer_utils import EvalPrediction
from torchvision.ops import box_convert
from dataclasses import dataclass


@dataclass
class ModelOutput:
    logits: torch.Tensor
    pred_boxes: torch.Tensor


def de_normalize_boxes(boxes, height, width):
    # 1. cxcywh → xyxy
    boxes_xyxy_norm = box_convert(boxes, 'cxcywh', 'xyxy')

    # 2. de-normalize (convert to actual pixel coordinates)
    boxes_xyxy_norm[:, [0, 2]] *= width
    boxes_xyxy_norm[:, [1, 3]] *= height
    return boxes_xyxy_norm


def map_compute_metrics(preprocessor=reference_preprocessor, threshold=0.0):
    map_metric = MeanAveragePrecision()
    post_process = preprocessor.post_process_object_detection

    def calc(eval_pred: EvalPrediction, compute_result=False):
        nonlocal map_metric

        if compute_result:
            m_ap = map_metric.compute()
            map_metric.reset()

            per_class_map = {
                f"{CLASSES[idx]}_mAP@0.50:0.95": m_ap.ap_per_class[idx].mean()
                for idx in m_ap.matched_classes
            }

            return {
                "mAP@0.50:0.95": m_ap.map50_95,
                "mAP@0.50": m_ap.map50,
                "mAP@0.75": m_ap.map75,
                **per_class_map
            }
        else:
            preds = ModelOutput(*eval_pred.predictions[1:3])
            labels = eval_pred.label_ids
            sizes = [label['orig_size'].cpu().tolist() for label in labels]

            results = post_process(preds, target_sizes=sizes, threshold=threshold)
            predictions = [Detections.from_transformers(result) for result in results]
            targets = [Detections(
                xyxy=de_normalize_boxes(label['boxes'], *label['orig_size']).cpu().numpy(),
                class_id=label['class_labels'].cpu().numpy(),
            ) for label in labels]

            map_metric.update(predictions=predictions, targets=targets)
            return {}
    return calc, map_metric

In [22]:
class DifferentiableLRTrainer(Trainer):
    def create_optimizer(self):
        backbone_params = []
        head_params = []

        for name, param in self.model.named_parameters():
            if 'backbone' in name:
                backbone_params.append(param)
            else:
                head_params.append(param)

        self.optimizer = torch.optim.AdamW([
            {'params': backbone_params, 'lr': self.args.backbone_lr},
            {'params': head_params, 'lr': self.args.learning_rate}
        ], weight_decay=self.args.weight_decay)

        return self.optimizer


class DifferentiableLRTrainingArguments(TrainingArguments):
    def __init__(self, *args, backbone_lr=None, **kwargs):
        super().__init__(*args, **kwargs)
        self.backbone_lr = backbone_lr

## Train

In [23]:
# Set Epoch Count & Learning Rate
EPOCHS = 20
REAL_BATCH = BATCH_SIZE[-1]
LEARNING_RATE = 1e-4

training_args = DifferentiableLRTrainingArguments(
    backbone_lr=LEARNING_RATE/10,  # Set backbone learning rate to 1/10th of the main learning rate
    learning_rate=LEARNING_RATE,
    lr_scheduler_type="cosine",
    warmup_ratio=0.1,
    weight_decay=0.1,
    max_grad_norm=0.5,
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE[0],
    per_device_eval_batch_size=BATCH_SIZE[1],
    gradient_accumulation_steps=REAL_BATCH//BATCH_SIZE[0],
    eval_accumulation_steps=BATCH_SIZE[1],
    batch_eval_metrics=True,
    remove_unused_columns=False,
    optim="adamw_torch",
    eval_strategy="steps",
    save_strategy="steps",
    logging_strategy="steps",
    eval_steps=100,
    save_steps=100,
    logging_steps=100,
    save_total_limit=100,
    load_best_model_at_end=True,
    metric_for_best_model="mAP@0.50:0.95",
    greater_is_better=True,
    #metric_for_best_model="eval_loss",
    #greater_is_better=False,
    #report_to="wandb",
    output_dir="./results/"+RUN_NAME,
    logging_dir="./logs/"+RUN_NAME,
    #run_name=RUN_NAME,
    bf16=True,
)

testing_args = TrainingArguments(
    per_device_eval_batch_size=BATCH_SIZE[2],
    batch_eval_metrics=True,
    remove_unused_columns=False,
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [24]:
from functools import partial

compute_metrics, compute_results = map_compute_metrics(preprocessor=reference_preprocessor)

trainer = DifferentiableLRTrainer(
    model=model,
    args=training_args,
    train_dataset=DatasetAdapterForTransformers(dataset.train),
    eval_dataset=DatasetAdapterForTransformers(dataset.valid),
    data_collator=partial(collate_fn, preprocessor=reference_preprocessor),
    compute_metrics=compute_metrics,
    #callbacks=[EarlyStoppingCallback(early_stopping_patience=30)]
)

tester = Trainer(
    model=model,
    args=testing_args,
    eval_dataset=DatasetAdapterForTransformers(dataset.test),
    data_collator=partial(collate_fn, preprocessor=reference_preprocessor),
    compute_metrics=compute_metrics
)

In [25]:
def start_train():
    accelerator = Accelerator()
    while True:
        try:
            try:
                print("INFO: Trying to resume from previous checkpoint")
                compute_results.reset()
                trainer.train(resume_from_checkpoint=True)
            except Exception as e:
                if "No valid checkpoint found" in str(e):
                    print(f"ERROR: Failed to resume from checkpoint - {e}")
                    print("INFO: Starting training from scratch")
                    compute_results.reset()
                    trainer.train(resume_from_checkpoint=False)
        except Exception as e:
            if "CUDA" in str(e):
                print(f"ERROR: CUDA Error - {e}")
                trainer.train()
            else:
                raise e

## Adaptation Testing

In [26]:
from safetensors.torch import load_file
from torch import nn

In [27]:
reference_config.return_dict = True  # Ensure the model returns a dictionary
model_pretrained = RTDetrForObjectDetection(config=reference_config)
model_states = load_file("RT-DETR_R50vd_SHIFT_CLEAR.safetensors", device="cpu")
model_pretrained.load_state_dict(model_states, strict=False)

for param in model_pretrained.parameters():
    param.requires_grad = False  # Freeze

model_pretrained

RTDetrForObjectDetection(
  (model): RTDetrModel(
    (backbone): RTDetrConvEncoder(
      (model): RTDetrResNetBackbone(
        (embedder): RTDetrResNetEmbeddings(
          (embedder): Sequential(
            (0): RTDetrResNetConvLayer(
              (convolution): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
              (normalization): RTDetrFrozenBatchNorm2d()
              (activation): ReLU()
            )
            (1): RTDetrResNetConvLayer(
              (convolution): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
              (normalization): RTDetrFrozenBatchNorm2d()
              (activation): ReLU()
            )
            (2): RTDetrResNetConvLayer(
              (convolution): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
              (normalization): RTDetrFrozenBatchNorm2d()
              (activation): ReLU()
            )
          )
          (pooler): MaxPool2d(

In [28]:
class FeatureNormalizationLayer(nn.Module):
    def __init__(self, target_dim=256):
        super().__init__()
        self.target_dim = target_dim

        # Keep only channel dimension
        self.adaptive_pool = nn.AdaptiveAvgPool2d((1, 1))

        # Linear compression
        self.linear_compress = nn.AdaptiveAvgPool1d(self.target_dim)

        # Feature normalization
        self.feature_norm = nn.Sequential(
            nn.LayerNorm(target_dim),
            nn.Dropout(0.1)
        )

    def forward(self, x):
        # Apply adaptive pooling
        x = self.adaptive_pool(x)

        # Squeeze channel dimension
        x = x.squeeze(-1).squeeze(-1)

        # Linear compression
        x = self.linear_compress(x)

        # Feature normalization
        x = self.feature_norm(x)

        return x

In [29]:
class APT(nn.Module):
    """
    Light-weight Sparse Autoencoder for Adaptation
    which learns how to sniff out the frame changes to predict next bounding boxes.
    """
    def __init__(self, feature_dim=256, bbox_dim=4, hidden_dim=32, sparsity_param=0.1):
        super().__init__()

        self.feature_dim = feature_dim
        self.bbox_dim = bbox_dim
        self.hidden_dim = hidden_dim
        self.sparsity_param = sparsity_param

        # Feature normalization layer for encoder-agnostic adaptation
        self.feature_norm = FeatureNormalizationLayer(target_dim=feature_dim)

        # Lightweight feature sniffer
        self.feature_sniffer = nn.Sequential(
            nn.Linear(feature_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim // 4 * 3)
        )

        # Previous bbox encoder
        self.bbox_encoder = nn.Sequential(
            nn.Linear(bbox_dim, hidden_dim // 4),
            nn.ReLU()
        )

        # Fusion layer
        self.fusion = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU()
        )

        # Prediction head
        self.predictor = nn.Sequential(
            nn.Linear(hidden_dim, bbox_dim),
            nn.Sigmoid()  # Normalize bbox coordinates to [0,1]
        )

        # Optional: Sparsity regularization
        self.activation = {}

    def forward(self, features, prev_bbox):
        # Normalize encoder features to be encoder-agnostic
        norm_features = self.feature_norm(features)

        # Extract relevant features from current frame
        sniffed_features = self.feature_sniffer(norm_features)

        # Encode previous bbox information
        bbox_features = self.bbox_encoder(prev_bbox)
        
        # 이미지 전체 특징(sniffed_features)을 각 bbox의 개수에 맞게 복제
        sniffed_features = sniffed_features.expand(bbox_features.shape[0], -1)

        # Fuse features
        fused = self.fusion(
            torch.cat([sniffed_features, bbox_features], dim=-1)
        )

        # Predict next bbox
        next_bbox = self.predictor(fused)

        # Store activation for sparsity regularization if needed
        self.activation['hidden'] = fused

        return next_bbox

    def get_sparsity_loss(self):
        """Calculate sparsity regularization loss"""
        if 'hidden' not in self.activation:
            return 0

        rho_hat = torch.mean(self.activation['hidden'], dim=0)
        rho = torch.full_like(rho_hat, self.sparsity_param)

        # KL divergence for sparsity regularization
        sparsity_loss = torch.sum(
            rho * torch.log(rho/rho_hat) +
            (1-rho) * torch.log((1-rho)/(1-rho_hat))
        )

        return sparsity_loss

In [30]:
from typing import Optional
from torchvision.ops import box_convert
from scipy.optimize import linear_sum_assignment

class TestTimeAdaptiveDETR(nn.Module):
    post_process = reference_preprocessor.post_process_object_detection
    
    def __init__(
        self, pretrained_model, img_size=(800, 1280),
        feature_dim=256, bbox_dim=4, hidden_dim=32, sparsity_param=0.1,
        confidence_threshold=0.7  # 신뢰도 임계값 추가
    ):
        super().__init__()
        self.model = pretrained_model
        self.apt = APT(
            feature_dim=feature_dim, bbox_dim=bbox_dim,
            hidden_dim=hidden_dim, sparsity_param=sparsity_param
        )
        self.img_size = img_size
        # bbox를 [0,1] 범위로 정규화하는 람다 함수
        self.__bbox_normalize = lambda bbox: bbox / torch.tensor([img_size[1], img_size[0], img_size[1], img_size[0]], device=bbox.device)
        self.bbox_cache = None  # 이제 {'boxes': tensor, 'scores': tensor} 형태의 dict로 관리
        self.adapt = False
        self.confidence_threshold = confidence_threshold

    def forward(
        self,
        pixel_values: torch.FloatTensor,
        pixel_mask: Optional[torch.LongTensor] = None,
        labels: Optional[list[dict]] = None,
        **kwargs,
    ):
        # 1. 기본 DETR 모델 실행 (인코더 특징 + 최종 예측 얻기)
        # return_dict=True와 output_hidden_states=True가 모델 config에 설정되어 있어야 함
        output = self.model(
            pixel_values=pixel_values,
            pixel_mask=pixel_mask,
            labels=labels,
            output_hidden_states=True,
            return_dict=True,
            **kwargs
        )
        
        apt_loss = None
        
        # --- 적응 로직 시작 (self.adapt가 True일 때만 동작) ---
        if self.adapt and self.bbox_cache is not None:
            # 2. 이전 프레임의 '신뢰도 높은' bbox만 필터링 (교사 신호 정제)
            prev_boxes = self.bbox_cache['boxes']
            prev_scores = self.bbox_cache['scores']
            
            high_conf_mask = prev_scores > self.confidence_threshold
            confident_prev_boxes = prev_boxes[high_conf_mask]
            
            # 신뢰도 높은 이전 박스가 있을 경우에만 적응 수행
            if confident_prev_boxes.shape[0] > 0:
                # 3. APT로 시간적 일관성에 기반한 '교사' 예측 수행
                features = output.encoder_last_hidden_state[-1]
                # APT 입력은 정규화된 cxcywh 포맷이어야 함
                normalized_prev_boxes_cxcywh = box_convert(self.__bbox_normalize(confident_prev_boxes), "xyxy", "cxcywh")
                apt_teacher_boxes = self.apt(features, normalized_prev_boxes_cxcywh) # 출력: cxcywh

                # 4. 현재 프레임의 '신뢰도 높은' 예측 필터링 (학습 대상 선별)
                # 모델의 최종 예측(logits, pred_boxes)은 아직 후처리를 거치지 않은 상태
                # pred_boxes는 정규화된 cxcywh 포맷
                student_boxes = output.pred_boxes[0] # 배치 크기는 1이라고 가정
                
                # 5. '교사'와 '학생' 예측을 매칭하여 손실 계산 (Hungarian Algorithm)
                with torch.no_grad():
                    # L1 거리 비용 매트릭스 계산
                    cost_matrix = torch.cdist(apt_teacher_boxes, student_boxes, p=1.0)
                    # 최적의 매칭 쌍 찾기
                    row_indices, col_indices = linear_sum_assignment(cost_matrix.cpu())

                # 매칭된 학생 예측(student_boxes)과 교사 예측(apt_teacher_boxes) 사이의 L1 Loss 계산
                matched_student_boxes = student_boxes[col_indices]
                apt_loss = nn.functional.l1_loss(matched_student_boxes, apt_teacher_boxes[row_indices])

        # --- 적응 로직 끝 ---

        # 6. 다음 프레임을 위해 현재 예측 결과를 bbox_cache에 저장
        if self.adapt:
            with torch.no_grad():
                sizes = [self.img_size for _ in range(len(output.pred_boxes))]
                # threshold를 낮게 설정하여 가능한 많은 후보를 캐시에 저장
                results = self.post_process(output, target_sizes=sizes, threshold=0.3)[0]
                self.bbox_cache = results # results는 {'scores': ..., 'labels': ..., 'boxes': ...} 형태

        # 7. 최종 손실 업데이트
        # 기존 loss (훈련 시) 또는 apt_loss (적응 시) 설정
        if labels is not None and hasattr(output, 'loss'): # 일반 훈련 시
            if apt_loss is not None:
                output.loss += apt_loss # 일반 훈련에도 apt_loss를 추가할 수 있음 (선택사항)
        else: # 적응 시
            output.loss = apt_loss if apt_loss is not None else torch.tensor(0.0, device=pixel_values.device)
        
        if apt_loss is not None:
            print(f"\rINFO: APT Loss - {apt_loss.item():.6f}", end="")

        return output

In [31]:
# Initialize Model
model = TestTimeAdaptiveDETR(pretrained_model=model_pretrained)
model.to(device)

TestTimeAdaptiveDETR(
  (model): RTDetrForObjectDetection(
    (model): RTDetrModel(
      (backbone): RTDetrConvEncoder(
        (model): RTDetrResNetBackbone(
          (embedder): RTDetrResNetEmbeddings(
            (embedder): Sequential(
              (0): RTDetrResNetConvLayer(
                (convolution): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
                (normalization): RTDetrFrozenBatchNorm2d()
                (activation): ReLU()
              )
              (1): RTDetrResNetConvLayer(
                (convolution): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
                (normalization): RTDetrFrozenBatchNorm2d()
                (activation): ReLU()
              )
              (2): RTDetrResNetConvLayer(
                (convolution): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
                (normalization): RTDetrFrozenBatchNorm2d()
                (activ

In [32]:
# Load Pretrained APT Weights
apt_weights = torch.load("./models/apt_model.pt")
model.apt.load_state_dict(apt_weights, strict=False)
for param in model.parameters():
    param.requires_grad = False  # Freeze APT weights
model.to(device)

TestTimeAdaptiveDETR(
  (model): RTDetrForObjectDetection(
    (model): RTDetrModel(
      (backbone): RTDetrConvEncoder(
        (model): RTDetrResNetBackbone(
          (embedder): RTDetrResNetEmbeddings(
            (embedder): Sequential(
              (0): RTDetrResNetConvLayer(
                (convolution): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
                (normalization): RTDetrFrozenBatchNorm2d()
                (activation): ReLU()
              )
              (1): RTDetrResNetConvLayer(
                (convolution): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
                (normalization): RTDetrFrozenBatchNorm2d()
                (activation): ReLU()
              )
              (2): RTDetrResNetConvLayer(
                (convolution): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
                (normalization): RTDetrFrozenBatchNorm2d()
                (activ

In [33]:
# Un-Freeze Model Encoder
for param in model.model.model.encoder.parameters():
    param.requires_grad = True  # Allow encoder to adapt during online adaptation

In [34]:
class NoShuffleTrainer(Trainer):
    def get_train_dataloader(self):
        return DataLoader(
            self.train_dataset,
            batch_size=self.args.per_device_train_batch_size,
            shuffle=False,  # Disable shuffling for online adaptation
            collate_fn=self.data_collator,
            num_workers=self.args.dataloader_num_workers,
            pin_memory=self.args.dataloader_pin_memory,
        )

In [35]:
from ttadapters.datasets import SHIFTContinuous100DatasetForObjectDetection

# Continuous Dataset
continuous_dataset = SHIFTContinuous100DatasetForObjectDetection(root=DATA_ROOT, train=True)

[08/24/2025 22:04:38] SHIFT DevKit - INFO - Base: ./data/SHIFT/continuous/images/100x/train. Backend: <shift_dev.utils.backend.ZipBackend object at 0x7fe5c4f18440>
[08/24/2025 22:04:38] SHIFT DevKit - INFO - Loading annotation from './data/SHIFT/continuous/images/100x/train/front/det_2d.json' ...


INFO: Downloading 'SHIFT' from file server to ./data/SHIFT/continuous/100x...
INFO: Dataset archive found in the root directory. Skipping download.


[08/24/2025 22:04:40] SHIFT DevKit - INFO - Loading annotation from './data/SHIFT/continuous/images/100x/train/front/det_2d.json' Done.
[08/24/2025 22:04:58] SHIFT DevKit - INFO - Loading annotation takes 20.11 seconds.


Batch 0:

Item                 Shape                               Min        Max       
--------------------------------------------------------------------------------
original_hw          [tensor([800]), tensor([1280])]
input_hw             [tensor([800]), tensor([1280])]
frame_ids            torch.Size([1])                           0.00       0.00
name                 ['00000000_img_front.jpg']
videoName            ['1b22-424f']
intrinsics           torch.Size([1, 3, 3])                     0.00     640.00
extrinsics           torch.Size([1, 4, 4])                  -172.22     340.81
boxes2d              torch.Size([1, 12, 4])                    0.00     702.00
boxes2d_classes      torch.Size([1, 12])                       0.00       1.00
boxes2d_track_ids    torch.Size([1, 12])                       0.00      11.00
images               torch.Size([1, 1, 3, 800, 1280])          0.00     255.00

Video name: 1b22-424f
Sample indices within a video: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10,

In [44]:
adapting_args = TrainingArguments(
    learning_rate=1e-6,
    lr_scheduler_type="cosine",
    warmup_ratio=0.1,
    weight_decay=0.1,
    max_grad_norm=0.5,
    num_train_epochs=1,
    per_device_train_batch_size=1,
    batch_eval_metrics=True,
    remove_unused_columns=False,
    optim="adamw_torch",
    save_strategy="epoch",
    logging_strategy="epoch",
    save_total_limit=100,
    load_best_model_at_end=False,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    output_dir="./adapts/"+RUN_NAME,
    logging_dir="./logs/"+RUN_NAME,
    #run_name=RUN_NAME,
    bf16=True,
    dataloader_drop_last=False
)

adapter = NoShuffleTrainer(
    model=model,
    args=adapting_args,
    train_dataset=DatasetAdapterForTransformers(continuous_dataset),
    data_collator=partial(collate_fn, preprocessor=reference_preprocessor),
    compute_metrics=compute_metrics
)

tester = Trainer(
    model=model,
    args=testing_args,
    eval_dataset=DatasetAdapterForTransformers(dataset.test),
    data_collator=partial(collate_fn, preprocessor=reference_preprocessor),
    compute_metrics=compute_metrics
)

revaluator = Trainer(
    model=model,
    args=testing_args,
    eval_dataset=DatasetAdapterForTransformers(dataset.valid),
    data_collator=partial(collate_fn, preprocessor=reference_preprocessor),
    compute_metrics=compute_metrics
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [45]:
# Check model
model.adapt = False
#revaluator.evaluate()

In [46]:
# Check shift
model.adapt = False
#tester.evaluate()

In [47]:
# Online Adaptation5
model.adapt = True
adapter.train()

INFO: APT Loss - 0.139769

Step,Training Loss


INFO: APT Loss - 0.186210

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



INFO: APT Loss - 0.191300

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



INFO: APT Loss - 0.141333

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



INFO: APT Loss - 0.170233

RuntimeError: 
            Some tensors share memory, this will lead to duplicate memory on disk and potential differences when loading them again: [{'model.class_embed.0.weight', 'model.model.decoder.class_embed.0.weight'}, {'model.model.decoder.class_embed.0.bias', 'model.class_embed.0.bias'}, {'model.model.decoder.class_embed.1.weight', 'model.class_embed.1.weight'}, {'model.model.decoder.class_embed.1.bias', 'model.class_embed.1.bias'}, {'model.class_embed.2.weight', 'model.model.decoder.class_embed.2.weight'}, {'model.model.decoder.class_embed.2.bias', 'model.class_embed.2.bias'}, {'model.model.decoder.class_embed.3.weight', 'model.class_embed.3.weight'}, {'model.model.decoder.class_embed.3.bias', 'model.class_embed.3.bias'}, {'model.model.decoder.class_embed.4.weight', 'model.class_embed.4.weight'}, {'model.model.decoder.class_embed.4.bias', 'model.class_embed.4.bias'}, {'model.class_embed.5.weight', 'model.model.decoder.class_embed.5.weight'}, {'model.class_embed.5.bias', 'model

In [48]:
model.model.save_pretrained("./models/RT-DETR_R50vd_SHIFT_ADAPT")

In [ ]:
# Evaluate Adapted Model for train domain (catastrophic forgetting)
model.adapt = False
revaluator.evaluate()